What we are developing here is called generative question-answering model using indexing library llama-index and text generation Bloom model.
- https://wandb.ai/mostafaibrahim17/ml-articles/reports/The-Answer-Key-Unlocking-the-Potential-of-Question-Answering-With-NLP--VmlldzozNTcxMDE3
- https://betterprogramming.pub/how-to-build-your-own-custom-chatgpt-with-custom-knowledge-base-4e61ad82427e

In [2]:
!pip install -U llama_index
!pip install -U transformers
!pip install panda
!pip install numpy
!pip install torch torchvision torchaudio
!pip install -U langchain
!pip install -U sentence-transformers

In [3]:
# Import necessary packages
# from llama_index import GPTSimpleVectorIndex, Document, SimpleDirectoryReader
import pandas as pd, numpy as np
import os, openai
from transformers import GPT2Tokenizer, GPT2LMHeadModel


# os.environ['OPENAI_API_KEY'] = ''
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
df = pd.read_json("./regItems.json")
df = df.replace(to_replace="", value=np.nan).dropna(axis=0) # remove null values
df['paragraphText'] = df['paragraphText'].str.replace("OLD SECTION.*", "", regex=True) # remove any dirty words
df['paragraphText'] = df['paragraphText'].str.replace("[a-zA-z]\d\w+", ". ", regex=True)
df['paragraphText'] = df['paragraphText'].str.lower()
df

,_id,chapter,article,title,paragraphText
2,{'$oid': '6417c99c2d49ca2fefed951e'},Chapter 17.5. Lead and Copper,Article 8. Lead Service Line Requirements for ...,§ 64688. Lead Service Line Replacement.,(a) a system shall replace lead service lines ...
3,{'$oid': '6417c99c2d49ca2fefed951f'},Chapter 17.5. Lead and Copper,Article 7. Public Education Program for Lead A...,§ 64687. Lead Public Education Program Content...,(a) each system with a lead action level excee...
4,{'$oid': '6417c99c2d49ca2fefed9520'},Chapter 17.5. Lead and Copper,Article 8. Lead Service Line Requirements for ...,§ 64689. Lead Service Line Sampling.,(a) each lead service line sample shall be one...
5,{'$oid': '6417c99d2d49ca2fefed9521'},Chapter 17.5. Lead and Copper,Article 6. Source Water Requirements for Actio...,§ 64686. Requirements Subsequent to the Depart...,(a) if the department determines that source w...
6,{'$oid': '6417c99e2d49ca2fefed9522'},"Chapter 15.5. Disinfectant Residuals, Disinfec...",Article 6. Reporting and Recordkeeping Require...,§ 64537.6. Disinfection Byproduct Precursors a...,(a) systems required to meet the enhanced coag...
...,...,...,...,...,...
759,{'$oid': '6417cac62d49ca2fefed9813'},Chapter 17. Surface Water Treatment,"Article 2. Treatment Technique Requirements, W...",§ 64653. Filtration.,(a) all approved surface water utilized by a s...
760,{'$oid': '6417cac62d49ca2fefed9814'},Chapter 17. Surface Water Treatment,"Article 2. Treatment Technique Requirements, W...",§ 64652. Treatment Technique Requirements and ...,(a) a supplier using an approved surface water...
761,{'$oid': '6417cac62d49ca2fefed9815'},Chapter 17. Surface Water Treatment,Article 3. Monitoring Requirements,§ 64655. Filtration Monitoring.,(a) to determine compliance with the performan...
762,{'$oid': '6417cac72d49ca2fefed9816'},Chapter 17. Surface Water Treatment,Article 3. Monitoring Requirements,"§ 64654.8. Source, Raw, Settled, and Recycled ...",(a) a supplier shall comply with the source mo...


In [5]:
data = df['paragraphText'].tolist()
# Prepare your data and convert it into a format that Llama Index can understand
# This example assumes that your data is a list of strings
formatted_data = [{'text': doc} for doc in data]

- https://github.com/jerryjliu/llama_index/blob/main/examples/test_wiki/TestNYC_Embeddings.ipynb
- https://gpt-index.readthedocs.io/en/latest/how_to/custom_llms.html#example-using-a-custom-llm-model

#### LLM customization with LlamaIndex

Note that we need to use the prompt helper to customize the prompt sizes, since every model has a slightly different context length.

Note that you may have to adjust the internal prompts to get good performance. Even then, you should be using a sufficiently large LLM to ensure it’s capable of handling the complex queries that LlamaIndex uses internally, so your mileage may vary.

A list of all default internal prompts is available here: 
- https://github.com/jerryjliu/llama_index/blob/main/gpt_index/prompts/default_prompts.py

Chat-specific prompts are listed here: 
- https://github.com/jerryjliu/llama_index/blob/main/gpt_index/prompts/chat_prompts.py


Llama-index usage pattern:
- https://github.com/jerryjliu/llama_index/blob/main/docs/guides/primer/usage_pattern.md

Custom LLM model:
- https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html#example-using-a-custom-llm-model

Custom Embeddings:
- https://gpt-index.readthedocs.io/en/latest/how_to/customization/embeddings.html

In [6]:
from langchain.llms.base import LLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, GPTSimpleVectorIndex, Document, PromptHelper, ServiceContext, LangchainEmbedding
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from IPython.display import Markdown
from typing import Optional, List, Mapping, Any

2023-04-14 20:02:36.161171: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-14 20:02:36.162611: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-14 20:02:36.162623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [7]:
# 2. Load the BigScience Bloomz model and tokenizer for text generation
model_name = "bigscience/bloom-560m" # "bigscience/bloomz"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, config='T5Config')

# Load the model from sentence transformers to create word embeddings
sentence_transformers_model = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hf_embeddings)

# define prompt helper that can help deal with token limitations
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [17]:
class CustomLLM(LLM):
    model_name = "bigscience/bloom-560m"
    
    # 3. Create the pipeline for question answering
    pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        # device=0, # GPU device number
        # max_length=256,
        # do_sample=True,
        # top_p=0.95,
        # top_k=50,
        # temperature=0.7
    )

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

In [18]:
#define our llm
llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper, embed_model=embed_model)

In [19]:
# 1. Load and index the dataset using Llama-index
index_path = "./index"


documents = [Document(d) for d in data]
# documents = []
# for text in data:
#   embedding = embed_model.get_text_embedding(text)
#   doc = Document(text, embedding=embedding)
#   documents.append(doc)


index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

index.save_to_disk('index.json')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 198292 tokens


In [20]:
new_index = GPTSimpleVectorIndex.load_from_disk('index.json', service_context=service_context)

In [21]:
# set Logging to DEBUG for more detailed outputs
response = new_index.query("what is cross-connection?", mode="default", verbose=True)
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Got node text: “system with a single service connection” means a system which supplies drinking water to consumers via a single service line....


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 317 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 6 tokens


<b>---------------------
The answer is “cross connection”. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is used in the context of a single service connection. Cross connection is a concept that is</b>

The bot response depends on the way we asked a question. See below for an example.

In [13]:
response = new_index.query("what is AWWA?", mode="default", verbose=True)
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Got node text: “surface water source augmentation project public water system” or “swsap pws” means a public water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water...


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 346 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 5 tokens


<b>---------------------
AWWA is a public water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter. The AWWA project is intended to provide an augmented reservoir to the public water system that is a source of drinking water for the residents of the city of Lake Charles. The project is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter. The project is expected to have an estimated cost of $25,000,000.
The question of what is the AWWA project?
The question of what is the AWWA project is answered as follows:
The AWWA project is a public water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter. The project is a public water system that plans to utilize an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter. The project is a public water system that plans to utilize an augmented reservoir as a source of drinking water and is responsible for compl</b>

In [15]:
response = new_index.query("what does AWWA abbreviation mean?", mode="default", verbose=True)
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Got node text: “surface water source augmentation project public water system” or “swsap pws” means a public water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water...


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 355 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 8 tokens


<b>---------------------
The word “surface water” is used in the context of water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter. The word “water system” is used in the context of a water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter.
The term “swapsap pws” means a public water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter. The term “water system” is used in the context of a water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter.
---------------------
In the context of a water system that plans to utilize or is utilizing an augmented reservoir as a source of drinking water and is responsible for complying with the requirements of chapter 17 and the applicable requirements of this chapter,</b>

In [ ]:
response = new_index.query("what does awwa abbreviation mean?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 333 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 9 tokens


<b>
AWWA stands for American Water Works Association.</b>

In [ ]:
response = new_index.query("what does the term State Board stands for?")
display(Markdown(f"<b>{response}</b>"))

<b>
The term "State Board" stands for the State Water Resources Control Board.</b>

In [ ]:
response = new_index.query("Give me a list of conditions on which the water system be removed from service.")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 350 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 16 tokens


<b>
1. The water system fails to meet the safety and quality standards set by the state board.
2. The water system fails to comply with the regulations and requirements set by the state board.
3. The water system fails to maintain the necessary infrastructure and equipment to provide safe and reliable service.
4. The water system fails to provide adequate customer service.
5. The water system fails to provide timely and accurate billing information.
6. The water system fails to provide timely and accurate maintenance and repair services.
7. The water system fails to provide timely and accurate water testing and monitoring services.
8. The water system fails to provide timely and accurate reporting of water quality data.
9. The water system fails to provide timely and accurate reporting of water usage data.
10. The water system fails to provide timely and accurate reporting of water system operations.
11. The water system fails to provide timely and accurate reporting of water system maintenance and repairs.
12. The water system fails to provide timely and accurate reporting of water system upgrades and improvements.
13. The water system fails to provide timely and accurate reporting of water system compliance with applicable laws and regulations.
14. The water system fails to provide timely and accurate reporting of water</b>

In [ ]:
response = new_index.query("I am not sure what the fecal coliform level is and can you explain what that is and how it can affect water system?")
display(Markdown(f"<b>{response}</b>"))

<b>
Fecal coliforms are a type of bacteria that are found in the intestines of warm-blooded animals, including humans. They are used as an indicator of water contamination by sewage or animal waste. High levels of fecal coliforms in a water system can indicate that the water is contaminated with disease-causing organisms, such as E. coli, which can cause serious illnesses.</b>

In [ ]:
response = new_index.query("What are some requirements an applicant should have before taking the T2 operator exam?")
display(Markdown(f"<b>{response}</b>"))

<b>
An applicant should have passed a Grade T1 operator examination within the three years prior to submitting the application for certification. They should also have completed at least one year of operator experience working as a certified T2 operator for a T2 facility or higher, or a facility that, prior to January 1, 2001, would have met the criteria for classification as a T2 facility or higher pursuant to Section 64413.1.</b>

In [ ]:
response = new_index.query("who is water user?")
display(Markdown(f"<b>{response}</b>"))

<b>
A water user is any individual or entity that uses water from a water system for human consumption. This includes residential customers, businesses, and other organizations.</b>

In [ ]:
response = new_index.query("can you explain what the water treatment facility and what functions it serves?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 276 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 14 tokens


<b>
A water treatment facility is a system of structures, equipment, and processes that are used to treat or condition a water supply. This treatment affects the physical, chemical, and bacteriological quality of the water, making it safe for public use. The facility may include processes such as filtration, sedimentation, disinfection, and other methods to remove contaminants and make the water safe for consumption. The facility may also include processes to adjust the pH, hardness, and other characteristics of the water. Facilities that only disinfect the water and do not require Giardia or virus reduction are not considered water treatment facilities.</b>

In [ ]:
response = new_index.query("what is the exact meaning of the water treatment facility?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 221 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 11 tokens


<b>
A water treatment facility is a group of structures, equipment, and processes that are used to improve the physical, chemical, or bacteriological quality of water that is distributed or offered to the public for domestic use by a public water system. Facilities that only disinfect water and are under the control of a certified distribution operator are not included as water treatment facilities.</b>

In [ ]:
response = new_index.query("What does DLR stand for?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 98 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens


<b>
DLR stands for "Detection Limit for Reporting".</b>

In [ ]:
response = new_index.query("What is DLR?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 123 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 5 tokens


<b>
DLR stands for "Detection Limit for Reporting" and is the designated minimum level at or above which any analytical finding of a contaminant in drinking water must be reported to the department.</b>

In [ ]:
response = new_index.query("What is a level 1 assessment?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 121 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens


<b>
A level 1 assessment is an evaluation to determine if there are any sanitary defects, defects in the distribution system coliform monitoring practices, and (when possible) the cause of the system triggering the assessment.</b>

In [ ]:
response = new_index.query("What is a level 2 assessment?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 207 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens


<b>
A level 2 assessment is an evaluation that provides a more detailed examination of a system than a level 1 assessment. It involves a comprehensive investigation and review of available information, additional internal and external resources, and other relevant practices to identify the possible presence of sanitary defects, defects in distribution system coliform monitoring practices, and (when possible) the likely reason that the system triggered the assessment.</b>

In [ ]:
response = new_index.query("What is the difference between a level 1 and 2 assessments?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 220 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens


<b>
The difference between a level 1 and 2 assessment is that a level 2 assessment provides a more detailed examination of the system than a level 1 assessment. This includes a more comprehensive investigation and review of available information, additional internal and external resources, and other relevant practices to identify the possible presence of sanitary defects, defects in distribution system coliform monitoring practices, and (when possible) the likely reason that the system triggered the assessment.</b>